In [28]:
import pandas as pd
import geohash2 as gh
from tqdm import tqdm
import json
from datetime import datetime

In [29]:
data = pd.read_csv('../model/gps_data2.csv')

def create_geohash(lat, lon):
	return gh.encode(lat, lon, precision=10)

In [30]:
data['Geohash'] = data.apply(lambda x: create_geohash(x['Latitude'], x['Longitude']), axis=1)

In [31]:
grouped_data = data.groupby('Geohash').agg({
	'Taxi ID': lambda x: list(set(x)),
	'Timestamp': list,
	'Latitude': 'mean',
	'Longitude': 'mean'
}).reset_index()

In [36]:
location_pairs = []

for _, row in tqdm(grouped_data.iterrows()):
	geohash = row['Geohash']
	person_ids = row['Taxi ID']
	timestamps = row['Timestamp']
	latitude = row['Latitude']
	longitude = row['Longitude']
	
	for i in range(len(person_ids)):
		for j in range(i + 1, len(person_ids)):
			pair = {
				'Geohash': geohash,
				'Person ID 1': person_ids[i],
				'Person ID 2': person_ids[j],
				'Timestamps': {
					person_ids[i]: timestamps[i],
					person_ids[j]: timestamps[j]
				},
				'Latitude': latitude,
				'Longitude': longitude
			}
			location_pairs.append(pair)

9418534it [05:50, 26880.25it/s]


In [37]:
with open('../files/edges/new_tdrive.json', 'w') as json_file:
	json.dump(location_pairs, json_file, indent=4)